# Image Study

On our first approach we will divide each tiktok in 10 frames and try to predict the virality from thgere with a CNN from scratch. We wil start preparing the data

In [1]:
from image_utils import remove_mp4_extensions, extract_frames, rename_frames_ids
import numpy as np
import pandas as pd
import os
import re
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from pathlib import Path
import sys
import PIL

In [2]:
root_dir = Path.cwd().resolve().parent.parent
sys.path.append(str(root_dir))
from config.variables import text_path, json_file, csv_file, textcsv_file

In [3]:
extract_frames()
# remove_mp4_extensions()

Saved: ../../data/inputs/frames\6875317312082201857_frame_0.png | Frame ID: 0
Saved: ../../data/inputs/frames\6875317312082201857_frame_1.png | Frame ID: 30
Saved: ../../data/inputs/frames\6875317312082201857_frame_2.png | Frame ID: 61
Saved: ../../data/inputs/frames\6875317312082201857_frame_3.png | Frame ID: 91
Saved: ../../data/inputs/frames\6875317312082201857_frame_4.png | Frame ID: 122
Saved: ../../data/inputs/frames\6875317312082201857_frame_5.png | Frame ID: 152
Saved: ../../data/inputs/frames\6875317312082201857_frame_6.png | Frame ID: 183
Saved: ../../data/inputs/frames\6875317312082201857_frame_7.png | Frame ID: 213
Saved: ../../data/inputs/frames\6875323773755657474_frame_0.png | Frame ID: 0
Saved: ../../data/inputs/frames\6875323773755657474_frame_1.png | Frame ID: 67
Saved: ../../data/inputs/frames\6875323773755657474_frame_2.png | Frame ID: 134
Saved: ../../data/inputs/frames\6875323773755657474_frame_3.png | Frame ID: 201
Saved: ../../data/inputs/frames\6875323773755657

In [3]:
# rename_frames_ids()

Renombrado: 6875317312082201857_frame_0.png a 6875317312082201857_frame_0.png
Renombrado: 6875317312082201857_frame_24.png a 6875317312082201857_frame_1.png
Renombrado: 6875317312082201857_frame_47.png a 6875317312082201857_frame_2.png
Renombrado: 6875317312082201857_frame_71.png a 6875317312082201857_frame_3.png
Renombrado: 6875317312082201857_frame_95.png a 6875317312082201857_frame_4.png
Renombrado: 6875317312082201857_frame_118.png a 6875317312082201857_frame_5.png
Renombrado: 6875317312082201857_frame_142.png a 6875317312082201857_frame_6.png
Renombrado: 6875317312082201857_frame_166.png a 6875317312082201857_frame_7.png
Renombrado: 6875317312082201857_frame_189.png a 6875317312082201857_frame_8.png
Renombrado: 6875317312082201857_frame_213.png a 6875317312082201857_frame_9.png
Renombrado: 6875323773755657474_frame_0.png a 6875323773755657474_frame_0.png
Renombrado: 6875323773755657474_frame_52.png a 6875323773755657474_frame_1.png
Renombrado: 6875323773755657474_frame_104.png a 6

In [6]:
from tensorflow.keras.preprocessing import image

def load_images(num_frames=8, img_height=224, img_width=224):
    dir = os.path.join(root_dir, 'data', 'inputs', 'frames')
    csv_path = os.path.join(root_dir, csv_file)
    df = pd.read_csv(csv_path, dtype={'id': str})
    X = []
    y = []

    # Iterar sobre cada fila del DataFrame
    for idx, row in df.iterrows():
        id_video = row['id']
        print(id_video)
        frames = []
        
        for j in range(num_frames):
            img_path = os.path.join(dir, f"{id_video}_frame_{j}.png")
            if os.path.exists(img_path):
                img = image.load_img(img_path, target_size=(img_height, img_width))
                img_array = image.img_to_array(img)
                frames.append(img_array)

        if len(frames) == num_frames:
            X.append(frames)
            y.append(row['norm_virality'])

    X = np.array(X) / 255.0
    y = np.array(y)

    return X, y

In [7]:
X, y = load_images()


6907228749016714497
6875468410612993286
6898699405898059010
6902819837345533186
6905635666588192002
6895497835681287426
6895303013867539713
6884590643327290625
6906514963569888513
6894206452450217217
6902751117805899009
6886417103046380801
6883211681988627714
6892769736589888770
6906571896683285761
6907334602017344774
6883094233935252737
6906818204966915330
6876318382703070465
6900948690186407170
6875323773755657474
6884935459110178049
6892805816492084481
6886765050355289346
6875370613523909890
6890189236130073857
6901321371113032962
6893611785295793410
6904536120755768578
6888873392234056962
6879952519586450690
6884933059758755073
6895846066961337601
6884933447480003842
6883604590525156609
6886215675350666497
6899411430660001026
6898613799402753281
6885861410329332998
6879772200065436929
6893119957534510338
6881624031561354498
6896541657471241474
6884688821976354049
6906870391466216709
6901020148426099969
6898035916682710274
6901681280128208130
6887645369895898369
6905029441760578818


In [8]:
X.shape

(1000, 8, 224, 224, 3)

In [9]:
X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=0.20, random_state=42)

X_test, X_valid, y_test, y_valid = train_test_split(X_test_valid, y_test_valid, test_size=0.50, random_state=42)
print("Tamaño del conjunto de entrenamiento:", len(X_train))
print("Tamaño del conjunto de validación:", len(X_valid))
print("Tamaño del conjunto de test:", len(X_test))

Tamaño del conjunto de entrenamiento: 800
Tamaño del conjunto de validación: 100
Tamaño del conjunto de test: 100


In [10]:
# Definir el modelo
model = models.Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(8, 224, 224, 3)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(Flatten()),
    layers.LSTM(64), 
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

c:\Users\ignac\Desktop\ICAI\TikTok-sentiment-analysis\tiktok\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.fit(X_train, y_train, epochs=1, batch_size=16, validation_data=(X_test, y_test))


50/50 ━━━━━━━━━━━━━━━━━━━━ 355s 7s/step - loss: 0.4319 - mae: 0.3193 - val_loss: 1.1519e-04 - val_mae: 0.0027


In [12]:
model.save_weights('8framesCNN.weights.h5')

In [13]:
#load weights
model.load_weights('8framesCNN.weights.h5')

In [14]:
train_mse, train_mae = model.evaluate(X_train, y_train, verbose=0)

# Evaluación en el conjunto de validación
valid_mse, valid_mae = model.evaluate(X_valid, y_valid, verbose=0)

In [15]:
train_rmse = np.sqrt(train_mse)
valid_rmse = np.sqrt(valid_mse)

In [16]:
print("Training Metrics:")
print("RMSE:", train_rmse)
print("MSE:", train_mse)
print("MAE:", train_mae)

print("\nValidation Metrics:")
print("RMSE:", valid_rmse)
print("MSE:", valid_mse)
print("MAE:", valid_mae)

Training Metrics:
RMSE: 0.019235054068572656
MSE: 0.0003699873050209135
MAE: 0.003626070683822036

Validation Metrics:
RMSE: 0.10009287380796127
MSE: 0.01001858338713646
MAE: 0.011656389571726322
